Goal: Investigate different combination of landmarks

Ex: glabella and chin, etc ...

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline
from itertools import chain, combinations
from tqdm import tqdm


vhr.plot.VisualizeParams.renderer = 'vscode' 

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
# -- LOAD A DATASET

dataset_name = 'mr_nirp'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

videos = constants.get_video_settings(dataset_name)
print(videos)

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

{'MOTION': [0, 2, 4, 6, 9, 11, 13], 'STILL': [1, 3, 5, 7, 8, 10, 12], 'MALE': [0, 1, 4, 5, 6, 7, 8, 9, 10, 13, 14], 'FEMALE': [2, 3, 11, 12], 'BEARD': [0, 1, 4, 5, 8, 9, 10, 13, 13], 'DARK': [0, 1, 2, 3, 4, 5, 13, 14]}
0 D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
2 D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\RGB_corrected\Subject2_motion_940.avi
3 D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\RGB_corrected\Subject2_still_940.avi
4 D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\RGB_corrected\Subject3_motion_940.avi
5 D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\RGB_corrected\Subject3_still_940.avi
6 D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\RGB_corrected\Subject4_motion_94

In [3]:
# -- PARAMETER SETTING

roi = 'nose'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 1     # minimum number of landmarks tested
max_len = 1
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for mr_nirp is 60 
Testing ROI = nose with minimum 1 landmarks


# Combinations

In [4]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [5]:
all_landmarks = vhr.extraction.utils.CustomLandmarks().get_all_landmarks()
elements = list(all_landmarks.keys()) # take all landmarks
# elements = list(rois[roi]) # take only landmarks in the ROI
all_combinations = list(chain.from_iterable(combinations(elements, r) for r in range(min_len,max_len+1)))
all_combinations = [list(i) for i in all_combinations]
print(f"For min length {min_len}, number of all combinations is {len(all_combinations)}")
# print(all_combinations)

valid_combinations = []
for comb in all_combinations:
    valid = True
    if 'left' in ''.join(comb):
        lefts = [s for s in comb if 'left' in s]
        for elem in lefts:
            if '_'.join(['right']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if 'right' in ''.join(comb):
        rights = [s for s in comb if 'right' in s]
        for elem in rights:
            if '_'.join(['left']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if valid:
        valid_combinations.append(comb)
print(f"For min length {min_len}, number of valid combinations is {len(valid_combinations)}")

df_combs = pd.DataFrame()
df_combs['names'] = valid_combinations
df_combs['names'] = df_combs['names'].apply(lambda x: tuple(x))
# get the corresponding landmarks values and remove duplicates
df_combs['values'] = df_combs['names'].apply(lambda x: list(set(list(chain.from_iterable( [all_landmarks[elem] for elem in x])))))

print(f"Number of possible combination of landmarks with {min_len} ROI (considering symmetry)", df_combs.shape)
df_combs


For min length 1, number of all combinations is 28
For min length 1, number of valid combinations is 8
Number of possible combination of landmarks with 1 ROI (considering symmetry) (8, 2)


,names,values
0,"(lower_medial_forehead,)","[10, 108, 109, 337, 338, 151]"
1,"(glabella,)","[8, 107, 108, 336, 337, 55, 151, 285]"
2,"(upper_nasal_dorsum,)","[193, 417, 419, 196, 197, 8, 55, 122, 285, 351]"
3,"(lower_nasal_dorsum,)","[3, 196, 5, 197, 419, 51, 248, 281]"
4,"(soft_triangle,)","[1, 4, 134, 457, 363, 44, 237, 45, 274, 275, 4..."
5,"(nasal_tip,)","[4, 5, 45, 275, 51, 281]"
6,"(philtrum,)","[0, 97, 2, 37, 326, 167, 393, 267]"
7,"(chin,)","[32, 194, 418, 262, 313, 140, 176, 400, 18, 83..."


In [6]:
# x = pd.read_hdf(f'../results/test_landmarks/h5/MR_NIRP/MR_NIRP_each_landmark.h5')
# valid_combinations = [list(a) for a in x['landmarks'].unique()]

valid_combinations = [('left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip'),
       ('chin', 'left_marionette_fold', 'right_marionette_fold'),
       ('left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar'),
       ('glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'),
       ('left_ala', 'left_lower_nasal_sidewall', 'left_mid_nasal_sidewall', 'lower_nasal_dorsum', 'nasal_tip', 'right_ala', 'right_lower_nasal_sidewall', 'right_mid_nasal_sidewall', 'soft_triangle', 'upper_nasal_dorsum'),
       ('left_temporal', 'right_temporal')]
print(len(valid_combinations))

6


In [7]:
class CombineLandmarks:
    def __init__(self, videos, dataset, winsize, patch_size, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.losses = []
        self.load_ground_truth(videos)

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def process(self,landmarks):
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            res = self.pipeline.run_on_video_multimethods(
                    ldmks_list=landmarks, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], 
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        # print(f"Total loss for {landmarks}: {sum(losses)}")
        self.losses.append(losses)

    def fit(self, landmarks_list):
        for landmarks in tqdm(landmarks_list):
            # print("Processing landmarks: ", landmarks)
            self.process(landmarks)

In [8]:
class CombineLandmarks:
    def __init__(self, videos, dataset, winsize, patch_size, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.losses = []
        self.load_ground_truth(videos)

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def process(self,landmarks):
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            res = self.pipeline.run_on_video_dtw(
                    ldmks_list=landmarks, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], sigGT=self.sigGT[i],
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        # print(f"Total loss for {landmarks}: {sum(losses)}")
        self.losses.append(losses)

    def fit(self, landmarks_list):
        for landmarks in tqdm(landmarks_list):
            # print("Processing landmarks: ", landmarks)
            self.process(landmarks)

In [10]:
pl = custom_pipeline.LandmarksPipeline()
np.arange(0, len(dataset.videoFilenames))
model = CombineLandmarks(np.arange(0, len(dataset.videoFilenames)), dataset, wsize, patch_size, pl, methods=['cupy_CHROM'], verb=False)
# model = CombineLandmarks([0], dataset, wsize, patch_size, pl, methods=['cupy_CHROM'], verb=False)
model.fit(valid_combinations)

Video name:  {15} ['D:/datasets_rppg/MR-NIRP_indoor\\Subject1_motion_940\\Subject1_motion_940\\RGB_corrected\\Subject1_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject1_still_940-015\\Subject1_still_940\\RGB_corrected\\Subject1_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject2_motion_940\\Subject2_motion_940\\RGB_corrected\\Subject2_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject2_still_940-002\\Subject2_still_940\\RGB_corrected\\Subject2_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject3_motion_940\\Subject3_motion_940\\RGB_corrected\\Subject3_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject3_still_940-012\\Subject3_still_940\\RGB_corrected\\Subject3_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject4_motion_940\\Subject4_motion_940\\RGB_corrected\\Subject4_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject4_still_940-004\\Subject4_still_940\\RGB_corrected\\Subject4_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject

 83%|████████▎ | 5/6 [1:37:26<19:29, 1169.39s/it]

Error:  setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (173,) + inhomogeneous part.


UnboundLocalError: local variable 'CC' referenced before assignment

In [ ]:
model.res[['DTW','normDTW', 'PPG_PCC']]

,DTW,normDTW
0,"[81.21330283653162, 80.87843717958461, 80.1671...","[0.1691943809094409, 0.1684967441241346, 0.167..."
0,"[72.55366335097986, 76.4738077547175, 79.10564...","[0.15115346531454138, 0.15932043282232813, 0.1..."
0,"[50.66701031959662, 64.0377569190572, 71.97939...","[0.10555627149915962, 0.13341199358136918, 0.1..."
0,"[45.688513615625745, 35.878468951285015, 34.26...","[0.09518440336588697, 0.07474681031517712, 0.0..."
0,"[66.180645307385, 65.47463195001953, 65.526168...","[0.13787634439038543, 0.13640548322920734, 0.1..."
...,...,...
0,"[51.42675574205933, 54.054195508405364, 54.452...","[0.10713907446262359, 0.11261290730917785, 0.1..."
0,"[76.70159131522821, 77.3274530295192, 78.00168...","[0.15979498190672545, 0.16109886047816502, 0.1..."
0,[nan],[nan]
0,"[66.31209496291181, 67.07440607693582, 66.8775...","[0.1381501978393996, 0.1397383459936163, 0.139..."


In [11]:
df = model.res.reset_index(drop=True)
print(df.columns)
df.head(1)

Index(['method', 'dataset', 'videoIdx', 'sigFilename', 'videoFilename', 'RMSE',
       'MAE', 'PCC', 'CCC', 'SNR', 'MAX', 'MAD', 'DTW', 'normDTW', 'bpmGT',
       'bpmES', 'bpmES_mad', 'timeGT', 'timeES', 'TIME_REQUIREMENT',
       'landmarks'],
      dtype='object')


,method,dataset,videoIdx,sigFilename,videoFilename,RMSE,MAE,PCC,CCC,SNR,...,MAD,DTW,normDTW,bpmGT,bpmES,bpmES_mad,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,,,,D:/datasets_rppg/MR-NIRP_indoor\Subject1_motio...,[2.3605563248129653],[1.4752155172413792],[0.7630136119267074],[0.755791507843119],[-1.6499751102598785],...,"[4.833984375, 6.591796875, 4.833984375, 3.0761...","[81.21330283653162, 80.87843717958461, 80.1671...","[0.1691943809094409, 0.1684967441241346, 0.167...","[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[69.43359375, 74.70703125, 78.662109375, 79.54...",,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",31.683129,"(left_nasolabial_fold, left_upper_lip, philtru..."


In [12]:
df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
df['videoIdx'] = df['videoFilename'].apply(lambda x: allvideo.index(x))
if dataset_name == 'lgi_ppgi':
  df['videoFilename'] = df['videoFilename'].apply(lambda x: x.split('\\')[2])
if dataset_name == 'mr_nirp':
  df['videoFilename'] = df['videoFilename'].apply(lambda x: x.split('\\')[2])
df['dataset'] = f'{dataset_name}'
indexes = {}
for v in range(len(allvideo)):
  indexes[allvideo[v].split('\\')[2]] = v
indexes = pd.DataFrame({'videoIdx':list(indexes.values()), 'videoFilename':list(indexes.keys())})
df = df.drop(columns=['sigFilename', 'bpmES_mad', 'videoIdx'])
df.insert(3, 'videoIdx', df['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

# from fastdtw import fastdtw
# df.insert(11, 'DTW', None)
# for row in df.itertuples():
#   distance, path = fastdtw(row.bpmGT, row.bpmES)
#   df.loc[row.Index, 'DTW'] = [distance]

print(df.shape, df.landmarks.unique().size)
df.head(1)

(90, 19) 6


,method,dataset,videoFilename,videoIdx,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,DTW,normDTW,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,mr_nirp,Subject1_motion_940,0,[2.3605563248129653],[1.4752155172413792],[0.7630136119267074],[0.755791507843119],[-1.6499751102598785],[9.51171875],"[4.833984375, 6.591796875, 4.833984375, 3.0761...","[81.21330283653162, 80.87843717958461, 80.1671...","[0.1691943809094409, 0.1684967441241346, 0.167...","[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[69.43359375, 74.70703125, 78.662109375, 79.54...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",31.683129,"(left_nasolabial_fold, left_upper_lip, philtru..."


In [13]:
df.landmarks.unique()

array([('left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip'),
       ('chin', 'left_marionette_fold', 'right_marionette_fold'),
       ('left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar'),
       ('glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'),
       ('left_ala', 'left_lower_nasal_sidewall', 'left_mid_nasal_sidewall', 'lower_nasal_dorsum', 'nasal_tip', 'right_ala', 'right_lower_nasal_sidewall', 'right_mid_nasal_sidewall', 'soft_triangle', 'upper_nasal_dorsum'),
       ('left_temporal', 'right_temporal')], dtype=object)

In [14]:
# print("saving into ", f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_combine_landmarks.h5')
# df.to_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_combine_landmarks.h5', key='df', mode='w')

print("saving into ", f'../results/test_landmarks/{dataset_name.upper()}_combine_landmarks_nose_missing.h5')
df.to_hdf(f'../results/landmarks/{dataset_name.upper()}/{dataset_name.upper()}_all_landmakrs_in_roi.h5', key='df', mode='w')

# df_old = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
# df_new = pd.concat([df_old, df], ignore_index=True)
# df_new.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')


saving into  ../results/test_landmarks/MR_NIRP_combine_landmarks_nose_missing.h5


C:\Users\erolland\AppData\Local\Temp\3\ipykernel_25948\80055103.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'MAD', 'DTW', 'normDTW', 'bpmGT', 'bpmES', 'timeGT',
       'timeES', 'landmarks'],
      dtype='object')]

  df.to_hdf(f'../results/landmarks/{dataset_name.upper()}/{dataset_name.upper()}_all_landmakrs_in_roi.h5', key='df', mode='w')


In [20]:
x = pd.read_hdf(f'../results/landmarks/{dataset_name.upper()}/{dataset_name.upper()}_each_landmakrs19.h5', key='df')
print("Number of videos", x.videoIdx.unique().size, x.videoFilename.unique())
print("Number of landmarks, ",x.landmarks.unique().size)
x.head(1)

Number of videos 15 ['Subject1_motion_940' 'Subject1_still_940' 'Subject2_motion_940'
 'Subject2_still_940' 'Subject3_motion_940' 'Subject3_still_940'
 'Subject4_motion_940' 'Subject4_still_940' 'Subject5_still_940'
 'Subject6_motion_940' 'Subject6_still_940' 'Subject7_motion_940'
 'Subject7_still_940' 'Subject8_motion_940' 'Subject8_still_940']
Number of landmarks,  18


,method,dataset,videoFilename,videoIdx,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,DTW,normDTW,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,mr_nirp,Subject1_motion_940,0,[1.974431501312688],[1.0424299568965518],[0.8106410267062507],[0.7909698467225462],[1.7849326349297088],[9.958984375],"[0.87890625, 1.318359375, 0.439453125, 0.43945...","[75.8044090459024, 72.44251910503768, 73.15707...","[0.15792585217896335, 0.15092191480216183, 0.1...","[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[68.5546875, 76.025390625, 79.541015625, 79.98...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",30.422719,"(upper_nasal_dorsum,)"
